In [1]:
import cv2
import numpy as np

In [2]:
def my_filtering(img, kernel):
    '''
    :param img: Image to apply the Gaussian filter
    :param kernel: Gaussian Kernel to apply the image
    :return: gaussian filtering is done using the input kernel.
    '''
    row = len(img)
    col = len(img[0])
    ksizeX = kernel.shape[1]
    ksizeY = kernel.shape[0]
    pad_image = cv2.copyMakeBorder(img, ksizeY//2, ksizeY // 2, ksizeX // 2, ksizeX // 2, cv2.BORDER_WRAP)
    filtered_img = np.zeros((row, col), dtype=np.float32) # Due to processing the negetive number and decimal point, declare the float.
    for i in range(row):
        for j in range(col):
            filtered_img[i, j] = np.sum(np.multiply(pad_image[i: i+ksizeY, j: j+ksizeX], kernel))

    return filtered_img

In [3]:
def find_zerocrossing(LoG, thresh = 0.01):
    '''
    :param LoG: Image filtered by LoG
    :param thresh: Due to error by float number's operation, use the thresh value.
    :return: Image in which only zero-crossing points have 255 values.
    '''
    y, x = len(LoG), len(LoG[0])
    res = np.zeros((y, x), dtype=np.uint8)
    for i in range(1, y - 1):  # Exclude the first and last (not to cross the border in image)
        for j in range(1, x - 1):
            near = LoG[i - 1: i + 1, j - 1: j + 1]
            max_value = np.max(near)
            min_value = np.min(near)
            if max_value * min_value * thresh < 0:
                res[i, j] = 255

    return res

In [4]:
def my_LoG(img, ksize=7):  # default sigma =1, sigma = 0.3(n/2 -1) + 0.8
    '''
    :param img: Image for LoG edge detection.
    :param ksize: Kernel size. Using the ksize x ksize kernel.
    :return: Image found by the LoG's method.
    '''
    sigma = 0.3 * (ksize / 2 - 1) + 0.8
    arr = np.arange(-ksize // 2 + 1., ksize // 2 + 1.)
    x, y = np.meshgrid(arr, arr)

    kernel = np.exp( -(x**2 + y**2) / (2 * sigma**2)) * (-1 + (x**2 + y**2) / 2 / sigma**2) * (1 / (np.pi * sigma**4))
    LoG_img = my_filtering(img, kernel) # LoG is a kernel
    LoG_img = find_zerocrossing(LoG_img)
    return LoG_img.astype(np.uint8)

In [5]:
def my_DoG(img, ksize, sigma, gx): #default (3,3) sigma = 1, partial differential of y axis
    '''
    :param img: Image for DoG edge detection.
    :param ksize: Kernel size. Using the ksize x ksize kernel.
    :param sigma: Standard deviation for the Gaussian distribution.
    :param gx: 0 is a partial differential of y axis and 1 is about x axis
    :return: Gradient value
    '''
    arr = np.arange(-ksize // 2 + 1., ksize // 2 + 1.)
    x, y = np.meshgrid(arr, arr)

    if gx is 0:
        kernel = np.exp(-(y**2) / (2. * sigma**2)) * (-y / sigma**2)
    if gx is 1:
        kernel = np.exp(-(x**2) / (2. * sigma**2)) * (-x / sigma**2)

    DoG_img = my_filtering(img, kernel) # DoG is a kernel
    return DoG_img

In [ ]:
src = cv2.imread('./sample/Lena.tiff', 1)
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

LoG = my_LoG(gray, 15)
DoGX = my_DoG(gray, 5, sigma=3, gx=1)
DoGY = my_DoG(gray, 5, sigma=3, gx=0)
DoG = np.sqrt(DoGX**2 + DoGY**2)
DoG = DoG.astype(np.uint8)
cv2.imshow("DoG", DoG)
cv2.imshow("LoG", LoG)

cv2.waitKey()
cv2.destroyAllWindows()